In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import kagglehub
from datasets import load_dataset
import os
from typing import Dict, List, Set, Tuple
from tqdm.notebook import tqdm
import csv

In [ ]:
playlist_dir = kagglehub.dataset_download("andrewmvd/spotify-playlists")
playlist_file = None
for root, dirs, files in os.walk(playlist_dir):
    for file in files:
        if file.endswith('.csv'):
            playlist_file = os.path.join(root, file)
            break
playlists_df = pd.read_csv(
    playlist_file,
    names=['user_id', 'artistname', 'trackname', 'playlistname'],
    skiprows=1,
    quoting=csv.QUOTE_ALL,
    escapechar='\\',
    on_bad_lines='skip'
)

In [ ]:
spotify_tracks_df = pd.read_csv("hf://datasets/maharshipandya/spotify-tracks-dataset/dataset.csv")

In [ ]:
playlists_df.head()

In [ ]:
spotify_tracks_df.head()

In [ ]:
# prompt: sort the spotify_tracks_df based on popularity (and print that out)

# Sort the DataFrame by popularity in descending order
spotify_tracks_df_sorted = spotify_tracks_df.sort_values(by='popularity', ascending=False)

# Print the sorted DataFrame
spotify_tracks_df_sorted.head(30)

In [ ]:
playlists_df.isnull().sum()

In [ ]:
rows_with_nulls = playlists_df.isnull().any(axis=1).sum()
rows_with_nulls

In [ ]:
print(f"Percentage of rows with missing values: {(rows_with_nulls/len(playlists_df)*100):.2f}%")

In [ ]:
print(spotify_tracks_df.isnull().sum())

In [ ]:
rows_with_nulls = spotify_tracks_df.isnull().any(axis=1).sum()
rows_with_nulls

In [ ]:
print(f"Percentage of rows with missing values: {(rows_with_nulls/len(spotify_tracks_df)*100):.2f}%")

In [ ]:
playlists_df_clean = playlists_df.dropna()
print(f"Original rows: {len(playlists_df)}")
print(f"Rows after dropping nulls: {len(playlists_df_clean)}")

In [ ]:
spotify_tracks_df_clean = spotify_tracks_df.dropna()
print(f"Original rows: {len(spotify_tracks_df)}")
print(f"Rows after dropping nulls: {len(spotify_tracks_df_clean)}")

In [ ]:
playlists_df_clean = playlists_df.dropna().copy()

playlists_df_clean.loc[:, 'track_clean'] = playlists_df_clean['trackname'].str.lower().str.strip()
playlists_df_clean.loc[:, 'artist_clean'] = playlists_df_clean['artistname'].str.lower().str.strip()

spotify_tracks_df.loc[:, 'track_clean'] = spotify_tracks_df['track_name'].str.lower().str.strip()
spotify_tracks_df.loc[:, 'artists_clean'] = spotify_tracks_df['artists'].str.lower().str.strip()

In [ ]:
# Check for duplicates in playlists_df_clean
print(f"Duplicate rows in playlists_df_clean: {playlists_df_clean.duplicated().sum()}")
if playlists_df_clean.duplicated().sum() > 0:
    print(playlists_df_clean[playlists_df_clean.duplicated(keep=False)])

# Check for duplicates in spotify_tracks_df_clean
print(f"Duplicate rows in spotify_tracks_df_clean: {spotify_tracks_df_clean.duplicated().sum()}")
if spotify_tracks_df_clean.duplicated().sum() > 0:
    print(spotify_tracks_df_clean[spotify_tracks_df_clean.duplicated(keep=False)])


In [ ]:
merged_df = playlists_df_clean.merge(
    spotify_tracks_df,
    left_on=['track_clean', 'artist_clean'],
    right_on=['track_clean', 'artists_clean'],
    how='inner'
)

In [ ]:
print(f"Original playlist rows: {len(playlists_df_clean)}")
print(f"Original spotify tracks rows: {len(spotify_tracks_df)}")
print(f"Matched rows: {len(merged_df)}")
print(f"Percentage of playlist songs matched: {(len(merged_df)/len(playlists_df_clean)*100):.2f}%")

In [ ]:
print("\nFirst few matches:")
print(merged_df[['trackname', 'track_name', 'artistname', 'artists']].head())

In [ ]:
print("Unique songs in matched data:")
print(f"Unique song-artist pairs in matched data: {merged_df[['track_clean', 'artist_clean']].nunique().iloc[0]}")
print(f"Unique playlists with at least one matched song: {merged_df['playlistname'].nunique()}")

playlist_matches = merged_df.groupby('playlistname').size()
print("\nMatched songs per playlist statistics:")
print(playlist_matches.describe())

In [ ]:
merged_df.head()

In [ ]:
len(merged_df) , len(merged_df.groupby(['playlistname', "user_id"]).size())

In [ ]:
len(merged_df) , len(merged_df.groupby(['playlistname', "user_id"]).size())

In [ ]:
# prompt: plotted distribution of the number of songs in each group (grouped by playlistname and user_id)
import matplotlib.pyplot as plt


# Group by playlistname and user_id and count the number of songs
song_counts = merged_df.groupby(['playlistname', 'user_id']).size()

# Create the plot
plt.figure(figsize=(12, 6))
sns.histplot(song_counts, kde=True)
plt.title('Distribution of Number of Songs per Playlist and User')
plt.xlabel('Number of Songs')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# prompt: number of groups with more than 100 songs (use the songcount variable)

# Calculate the number of groups with more than 100 songs
num_groups_over_100 = sum(1 for count in song_counts if count > 1000)

print(f"Number of groups with more than 100 songs: {num_groups_over_100}")

In [ ]:
merged_df.iloc[:10, 11:27]

In [ ]:
# prompt: sort the spotify tracks dataset based on popularity

# Sort the merged DataFrame by popularity in descending order
merged_df_sorted = merged_df.sort_values(by='popularity', ascending=False)

# Display the sorted DataFrame (or work with it as needed)
print(merged_df_sorted.head())

In [ ]:

# Step 1: Drop playlists with fewer than 10 songs
# -----------------------------------------------
# Compute the counts of songs per playlist
playlist_counts = merged_df.groupby(['playlistname', 'user_id']).size().reset_index(name='counts')

# Filter playlists with at least 10 songs
valid_playlists = playlist_counts[playlist_counts['counts'] >= 10][['playlistname', 'user_id']]

# Merge back to the original DataFrame to keep only valid playlists
merged_df_valid = merged_df.merge(valid_playlists, on=['playlistname', 'user_id'], how='inner')

In [ ]:
len(merged_df_valid), len(merged_df)

In [ ]:
merged_df_valid.head()

In [ ]:
playlists = merged_df_valid[['playlistname', 'user_id']].drop_duplicates()

# Shuffle the playlists
playlists = playlists.sample(frac=1, random_state=42).reset_index(drop=True)

# Compute split indices
total_playlists = len(playlists)
train_end = int(0.8 * total_playlists)
val_end = int(0.9 * total_playlists)

# Assign playlists to train, validation, and test sets
playlists['set'] = ['train'] * train_end + ['val'] * (val_end - train_end) + ['test'] * (total_playlists - val_end)

# Merge the set assignments back to the main DataFrame
merged_df_valid = merged_df_valid.merge(playlists, on=['playlistname', 'user_id'], how='left')

In [ ]:
merged_df_valid.head()

In [ ]:
def assign_XY(group):
    # Shuffle the songs within the playlist
    group = group.sample(frac=1, random_state=42).reset_index(drop=True)
    total_songs = len(group)
    split_point = int(0.8 * total_songs)
    group['XY'] = ['X'] * split_point + ['Y'] * (total_songs - split_point)
    return group

# Apply the function to each set
merged_df_valid = merged_df_valid.groupby(['playlistname', 'user_id', 'set'], group_keys=False).apply(assign_XY)

In [ ]:
# Columns to keep as per your specification
columns_to_keep = [
    'playlistname',   # 1. The playlist it is from
    'user_id',        # 2. The user who created the playlist
    'track_clean',    # 3. The song name
    'artist_clean',   # 4. The artist name
    'album_name'      # 5. The album name
]

# Add the features from columns 11 to 27 (0-indexed)
# Assuming merged_df has the original columns from 0 to N
# Get the column names for columns 11 to 27
feature_columns = merged_df.columns[11:27].tolist()
columns_to_keep += feature_columns

# Add the 'XY' column for labels
columns_to_keep += ['XY']

# Now, filter the DataFrame to keep only these columns
merged_df_valid = merged_df_valid[columns_to_keep + ['set']]  # Include 'set' to separate datasets
# Create a unique song ID using 'track_clean' and 'artist_clean'
merged_df_valid['song_id'] = merged_df_valid['track_clean'] + ' - ' + merged_df_valid['artist_clean']

# Create a unique playlist ID using 'playlistname' and 'user_id'
merged_df_valid['playlist_id'] = merged_df_valid['playlistname'] + ' - ' + merged_df_valid['user_id']


In [ ]:
merged_df_valid.iloc[100:150]

In [ ]:
merged_df_valid.to_csv('data/full_data.csv', index=False)


In [ ]:
train_df = merged_df_valid[merged_df_valid['set'] == 'train'].drop(columns=['set'])
val_df = merged_df_valid[merged_df_valid['set'] == 'val'].drop(columns=['set'])
test_df = merged_df_valid[merged_df_valid['set'] == 'test'].drop(columns=['set'])


In [ ]:
train_df.to_csv('data/train.csv', index=False)
val_df.to_csv('data/val.csv', index=False)
test_df.to_csv('data/test.csv', index=False)

In [ ]:
train_df.head()